## Imports and Base models

In [1]:
from typing import Literal
from langchain import hub
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from dotenv import load_dotenv
import os
load_dotenv()

d:\02-NextCloud\github\LangChain-Playground\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
gemini_token = os.getenv("GEMINI_TOKEN")
structurated_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=gemini_token)

In [13]:
hf_token = os.getenv("HF_TOKEN")
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=hf_token
)
chat_model = ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\piamp\.cache\huggingface\token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Query Analysis

In [9]:
# Data Model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource"""

    datasource: Literal["vectorstore", "duckduckgo", "github", 'notify', 'none'] = Field(
        ...,
        description="Given a user question choose to route to the most relevant datasource"
    )

LLM_router = structurated_model.with_structured_output(RouteQuery)

In [10]:
# Prompt
system = """You are PampuAI, an expert at routing a user question to different tools.
The 'vectorstore' contains documents, including martin's cv and all actions that you can perform (PampuAI).
Use 'duckduckgo' for questions that can be answered by a web search.
Use 'github' for questions about martin's recent projects.
Use 'notify' to send a notification with a message to martin.
Use 'none' for chatting with PampuAI.
"""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | LLM_router

In [61]:
print(question_router.invoke({"question": "Que tecnologias maneja martin?"}))
print(question_router.invoke({"question": "Cual es el nombre de Obama?"}))
print(question_router.invoke({"question": "En que proyectos trabajo martin ultimamente?"}))
print(question_router.invoke({"question": "Decile a martin que me gusto su portfolio"}))
print(question_router.invoke({"question": "Que podes hacer?"}))
print(question_router.invoke({"question": "Hola!"}))

datasource='vectorstore'
datasource='duckduckgo'
datasource='github'
datasource='notify'
datasource='vectorstore'
datasource='none'


# Tools

## VectorStore

## DuckDuckGo

In [30]:
system = """
You are an assistant for question-answering tasks. Use only the provided context to answer the question.
Give the response in the same language as the question was asked.
"""

search_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Context: {context}\nQuestion: {question}"),
    ]
)

search = DuckDuckGoSearchResults()
def route_duckduckgo(question):
    results = search.invoke(question)
    chain = search_prompt | chat_model
    response = chain.invoke({"question": question, "context": results})
    return response.content

# Implementation 

Looking for a better implementation using langgraph and **langserve** !!!

In [29]:
while True:
    question = input("You: ")
    if question == "exit":
        break

    # Route the question
    router = question_router.invoke({"question": question})
    print("Selected tool:", router.datasource)

    if router.datasource == "notify":
        response = print("Sending notification...")

    elif router.datasource == "none":
        response = print("Chatting with PampuAI...")

    elif router.datasource == "vectorstore":
        response = print("Searching in vectorstore...")

    elif router.datasource == "duckduckgo":
        response = route_duckduckgo(question)

    elif router.datasource == "github":
        response = print("Searching in github...")

    print("PampuAI:", response)

Selected tool: duckduckgo
PampuAI: Según el artículo, el próximo Major de CS es el PGL CS2 Major Copenhague 2024, que se celebrará del 17 al 31 de marzo en Copenhague.
